In [1]:
##############################################################################
# OPTIMIZATION & AI - LAB 04   Pierre LOVITON
##############################################################################

In [148]:
# System imports
import matplotlib.pyplot as plt
import numpy as np

#Extra library imports
import pandas as pd
from scipy.optimize import linprog, brute
from itertools import product

In [3]:
# User inputs
csvfile1 = 'vote-v1.csv'
csvfile2 = 'vote-v2.csv'
max_budget = 500000

Participatory budgeting is a sourcing process allowing city or country to fund budgets based on the votes of the involved citizens. For example, the City Hall of Paris organizes a campaign every year since 2015, where Parisian can vote for projects supporting many various fields impacting their everyday life such as ecological transition, sport or education (see https://budgetparticipatif.paris.fr for precise examples)! In this lab, we will consider the selection of projects after the vote as an constrained optmization problem, and compare this approach with the one chosen until 2020.

# I.  Paris' participatory budgeting

## a) Data exploration

We will first work on a realistic dataset, based on samples of real projects submitted for voting.

<font color='blue'> Question 1: open the 'votes-v1.csv' using pandas. Assign a variable "nb_projects" with the number of projects, for example using the attribute "shape" from Pandas. Display the available information for the 5 first projects using the head attribute. </font>

In [177]:
# Load the first csv file
df = pd.read_csv('votes-v1.csv')

# Get the number of projects
nb_projects = df.shape[0]
print('{0} projects'.format(nb_projects))

# Display the 5 first lines
df.head()

12 projects


,Project_id,Nb_votes,Cost
0,proj-01,195,100000
1,proj-02,167,30000
2,proj-03,262,15000
3,proj-04,227,415000
4,proj-05,110,6000


In [8]:
df.shape

(12, 3)

We read for example that the cost of the first project is 100000 euros, and that it received 195 votes.

<font color='blue'> Question 2: supposing that each voter could choose only one project, how many people did vote? If all the projects were accepted, what would be the total cost? You may use the "sum" attribute from Pandas to answer these questions.</font> 

In [178]:
nb_total_voters = sum(df.Nb_votes)
print('{0} total votes'.format(nb_total_voters))

nb_total_cost = sum(df.Cost)
print('{0} total cost'.format(nb_total_cost))

1880 total votes
1319000 total cost


Supposing that the City Hall of Paris cannot spend more than 500000 euros to fund these projects, you should conclude that unfortunately all the projects cannot be selected.

Moreover, as in other democratic processes, we usually reason using the percentage of voters rather than the total number of voters. To that purpose, we now create a new variable.

<font color='blue'> Question 3: add the column 'Ratio_voters' associated with each project the ratio of voters who voted for that project.</font> 

In [179]:
df['Ratio_voters'] = round(df.Nb_votes/nb_total_voters*100,2)

In [16]:
df.head()

,Project_id,Nb_votes,Cost,Ratio_voters
0,proj-01,195,100000,10.37
1,proj-02,167,30000,8.88
2,proj-03,262,15000,13.94
3,proj-04,227,415000,12.07
4,proj-05,110,6000,5.85


<font color='blue'> Question 4: compute automatically and display the name of the project that received the highest percentage of votes. Compute automatically and display also the highest percentage of votes. You may use the max attribute from Pandas. </font> 

In [18]:
highest_percentage_votes = max(df.Ratio_voters)
print('{0}% highest percentage of votes'.format(highest_percentage_votes))

13.94% highest percentage of votes


<font color='blue'> Question 5: compute automatically and display the name of the project that received the lowest percentage of votes. Compute automatically and display also the lowest percentage of votes. You may use the min attribute from Pandas. </font> 

In [19]:
lowest_percentage_votes = min(df.Ratio_voters)
print('{0}% lowest percentage of votes'.format(lowest_percentage_votes))

3.99% lowest percentage of votes


## b) Project selection using linear programming

Once the votes have been received, the process should select projects to satisfy as many voters as possible. However, the City Hall of Paris limits the budget allocated to fund all the selected projects to a fix amount (the "max_budget" variable).
We formulate this decision making process as the following linear programming problem:

$\min \boldsymbol{-}\displaystyle \sum_{i=1}^n x_i r_i$ s.t. $\min \displaystyle \sum_{i=1}^n x_i c_i \leq \textrm{max_budget}$

with $n$ the number of projects, $x_i$ the decision of selecting or not project $i$, $r_i$ the percentage of voters choosing project $i$, and $c_i$ the cost of project $i$.
We would like $x$, the vector of variables to optimize, to represent the percentage of chance for a project to be selected. Thus, we force each variable $x_i$ to belong in $[0, 1]$.
We will now solve this linear programming problem using Scipy.

<font color='blue'> Question 6: complete the code below to solve this problem using the **revised simplex method**.</font> 

In [27]:
# Set the coefficients of the linear objective function to be minimized
c = -df['Ratio_voters'].values

# Set the inequality constraint matrix
A = [list(df.Cost.values),]

# Set the inequality constraint vector
max_budget = 500000
b = [max_budget]

#Set the bounds
bounds = []
for i in range (nb_projects):
    bounds.append((0,1))
    
# Select the optimization method
method = 'revised simplex'

# Perform the optimization using the simplex method
res = linprog(c, A_ub=A, b_ub=b, method=method, bounds=bounds)

# Display the optimization result
print(res)

     con: array([], dtype=float64)
     fun: -80.05125301204819
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([0.])
  status: 0
 success: True
       x: array([1.        , 1.        , 1.        , 0.20722892, 1.        ,
       1.        , 1.        , 1.        , 0.        , 1.        ,
       1.        , 1.        ])


<font color='blue'> Question 7: display the objective function value after optimization using the appropriated attribute of the optimization result. Compare this value with the one obtained using the "x" attribute of the optimization result and the appropriated input.</font> 

In [33]:
res

     con: array([], dtype=float64)
     fun: -80.05125301204819
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([0.])
  status: 0
 success: True
       x: array([1.        , 1.        , 1.        , 0.20722892, 1.        ,
       1.        , 1.        , 1.        , 0.        , 1.        ,
       1.        , 1.        ])

In [83]:
print("The optimal value of the objective function is",round(res.fun,2))
print("The x array shows us the probability of each project being chosen ", res.x)
print("If we divide multiply x by r and -1 and we do the sum we have to obtain f :", round(-1*np.sum(res.x*df['Ratio_voters']),2))

The optimal value of the objective function is -80.05
The x array shows us the probability of each project being chosen  [1.         1.         1.         0.20722892 1.         1.
 1.         1.         0.         1.         1.         1.        ]
If we divide multiply x by r and -1 and we do the sum we have to obtain f : -80.05


Visually, we see that each optimized variable value belongs in $[0, 1]$. However, we still have to decide to select or not the associated projects. To that purpose, we decide to select projects associated with an optimized variable value strictly higher than 0.5.

<font color='blue'> Question 8: select the projects associated with a final value strictly higher than 0.5. Display automatically the number of selected projects and their names. Are the most and the least popular projects selected? </font> 

In [73]:
proj_selected = df['Project_id'].loc[res.x>=0.5]
print(proj_selected)
print("\nNumber of project chosen is :",proj_selected.shape[0])

print(df['Project_id'].loc[df['Ratio_voters']==highest_percentage_votes])
print(df['Project_id'].loc[df['Ratio_voters']==lowest_percentage_votes])

0     proj-01
1     proj-02
2     proj-03
4     proj-05
5     proj-06
6     proj-07
7     proj-08
9     proj-10
10    proj-11
11    proj-12
Name: Project_id, dtype: object

Number of project chosen is : 10
2    proj-03
Name: Project_id, dtype: object
5    proj-06
Name: Project_id, dtype: object


<font color='blue'> 
    We can see that proj-03 (highest percentage votes) and proj-06 (lowest percentage votes) are projects selected
</font> 

<font color='blue'> Question 9: compute and display the total cost of all the finally selected projects by our method and the percentage of satisfied voters. Is the maximal allocated budget exceeded?</font> 

In [84]:
# Total cost of selected projects 
print("The total cost of all the finally selected projects is",sum(df['Cost'].loc[res.x>=0.5]))

# Total ratio of satisfied voters 
print("The percentage of satisfied voters is",sum(df['Ratio_voters'].loc[res.x>=0.5]))

The total cost of all the finally selected projects is 414000
The percentage of satisfied voters is 77.54999999999998


<font color='blue'> 
The allocated budget is 500 000 and we can see see that the total cost is 414 000 so it's not exceeded.
</font> 

<font color='blue'> Question 10: solve the problem using the **interior point method**. What is the main difference in performances between the two algorithms? What algorithm would you prefer for this application?</font> 

In [95]:
# Perform the interior point method
method = 'interior-point'
res = linprog(c, A_ub=A, b_ub=b, method=method, bounds=bounds)

# Display the optimization result
print(res)

     con: array([], dtype=float64)
     fun: -80.05125301209175
 message: 'The solution does not satisfy the constraints within the required tolerance of 3.16E-04, yet no errors were raised and there is no certificate of infeasibility or unboundedness. Check whether the slack and constraint residuals are acceptable; if not, consider enabling presolve, adjusting the tolerance option(s), and/or using a different method. Please consider submitting a bug report.'
     nit: 6
   slack: array([-0.00101738])
  status: 4
 success: False
       x: array([9.99999999e-01, 1.00000000e+00, 9.99999998e-01, 2.07228917e-01,
       1.00000000e+00, 9.99999996e-01, 9.99999996e-01, 1.00000000e+00,
       4.80723428e-12, 1.00000000e+00, 9.99999999e-01, 1.00000000e+00])


<font color='blue'> 
The main difference is that the interior point method has a higher complexity than the revised simplex. It's better to chose the second one for simple linear problem (2D). The interior point and revised simplex methods converge but the first one need more iterations.
    So here we prefer for this problem the revised simplex method.
</font> 

## c) Comparison with other methods

We will now compare our linear programming approach with the method initially applied by the City Hall of Paris. Until 2020, the projects were ordered by decreasing order of the number of votes, and progressively selected as long as the total cost value of the chosen projects remained less than or equal to the maximal allocated budget.

<font color='blue'> Question 11: implement the selection method applied by the City Hall of Paris until 2020. You may use the "sort_values" and "cumsum" values of Pandas. What are the names of the projects selected by this approach? Is the less popular project selected? Compute and display the total cost of the selected projects, and the percentage of satisfied voters. Compare with our method.</font> 

In [112]:
df = df.sort_values('Nb_votes',ascending= False).reset_index(drop=True)
df.head()

,Project_id,Nb_votes,Cost,Ratio_voters
0,proj-03,262,15000,13.94
1,proj-04,227,415000,12.07
2,proj-01,195,100000,10.37
3,proj-09,195,490000,10.37
4,proj-07,191,25000,10.16


In [129]:
df['Cost_cumsum'] = df.Cost.cumsum().reset_index(drop=True)
list1 = []
list2 = []

for i in range(df.shape[0]): 
    if df.Cost_cumsum[i] <=500000:
        list1.append(df['Project_id'][i])
        list2.append(df['Ratio_voters'][i])

    else:
        print("We cannot select another project, total cost is",df['Cost_cumsum'][i-1])
        print("We selected these projects:",list1)
        print("Percentage of satisfied voters is:",round(sum(list2),2))  
        break

We cannot select another project, total cost is 430000
We selected these projects: ['proj-03', 'proj-04']
Percentage of satisfied voters is: 26.01


<font color='blue'> With this method, we can see immediatly that there is a problem. The two projects with the most votes are selected, but only these two. Only 2 project were selected for 430000$ and 26.01% of satisfied voters, and with our method, there were 77.54% satisfied voters and a budget of 414000. Our method is really better than this one. </font> 

We will now compare the linear programming approach with the brute force method, i.e computing all the possible solutions and selecting the best one. **Ensure that the constraints are also respected!**

<font color='blue'> Question 12: what is the total number of possible solutions? Write the answer as a function of the number of projects $n$. Compute the best solution among all the possible ones. You may use the "product" function of the "itertool" library. Display the names, the total cost, and the percentage of voters satisfied by the selected projects. Did we obtain the best possible solution using linear programming?</font> 

In [180]:
list_sol =[]
list_ratio_voters = []
print("The number of total solution is:",2**12)

def bruteforce(n):
    for choice in product([0,1],repeat=n):
        sol = np.array((choice))
        df['solutions'] = sol
        df_choosen = df.loc[df['solutions']==1]
        if sum(df_choosen['Cost'])<=500000:
            #print(df_choosen)
            list_sol.append(sol)
            list_ratio_voters.append(sum(df_choosen['Ratio_voters']))
    best_ratio_voters = max(list_ratio_voters)
    index_ratio = list_ratio_voters.index(best_ratio_voters)
    best_sol = list_sol[index_ratio]
    df['solutions'] = best_sol
    return best_ratio_voters, best_sol

best_ratio_voters, best_sol = bruteforce(12)

The number of total solution is: 4096


In [184]:
print("The project selected, with the brute force method, are:",df[df['solutions']==1].Project_id)
print("\nThe total cost is:",df[df['solutions']==1].Cost.sum())
print("\nThe percentage of satisfied voters is:",best_ratio_voters)

The project selected, with the brute force method, are: 0     proj-01
1     proj-02
2     proj-03
4     proj-05
5     proj-06
6     proj-07
7     proj-08
9     proj-10
10    proj-11
11    proj-12
Name: Project_id, dtype: object

The total cost is: 414000

The percentage of voters is: 77.54999999999998


<font color='blue'> As we can see, the result of the total cost and the percentage of satisfied voters are the same. And the project rejected are the fourth one and the nineth one. If we compare, we can see that this is exactly the same result as our first method. We can conclude that our method with linprog and revised simplex was the best. </font> 

## d) Limits of our linear programming approach to select projects

<font color='blue'> Question 13: perform brute-force, our linear programming approach using either the simplex or the interior-point method, and the algorithm originally used by the City Hall of Paris to select projects on the **'votes-v2.csv'** file. Does any approach, brute force excluded, select the expected best solution? Is the maximal allocated budget exceeded after thresholding at 0.5 the optimized variables using linear programming? What constraints should be added to properly solve this problem? </font> 

In [189]:
# Load the first csv file
df2 = pd.read_csv('votes-v2.csv')
df2

,Project_id,Nb_votes,Cost
0,proj-01,1010,54316
1,proj-02,369,443312
2,proj-03,1984,173569
3,proj-04,1461,433077
4,proj-05,581,430308


In [190]:
nb_total_voters2 = sum(df2.Nb_votes)
print('{0} total votes'.format(nb_total_voters2))

nb_total_cost2 = sum(df2.Cost)
print('{0} total cost'.format(nb_total_cost2))

5405 total votes
1534582 total cost


In [191]:
df2['Ratio_voters'] = round(df2.Nb_votes/nb_total_voters2*100,2)

In [192]:
df2

,Project_id,Nb_votes,Cost,Ratio_voters
0,proj-01,1010,54316,18.69
1,proj-02,369,443312,6.83
2,proj-03,1984,173569,36.71
3,proj-04,1461,433077,27.03
4,proj-05,581,430308,10.75


In [195]:
df2.Cost.values

array([ 54316, 443312, 173569, 433077, 430308])

In [214]:
# BRUTE FORCE METHOD
list_sol =[]
list_ratio_voters = []

def bruteforce(n):
    for choice in product([0,1],repeat=n):
        sol = np.array((choice))
        df2['solutions'] = sol
        df_choosen2 = df2.loc[df2['solutions']==1]
        if sum(df_choosen2['Cost'])<=500000:
            #print(df_choosen)
            list_sol.append(sol)
            list_ratio_voters.append(sum(df_choosen2['Ratio_voters']))
    best_ratio_voters = max(list_ratio_voters)
    index_ratio = list_ratio_voters.index(best_ratio_voters)
    best_sol = list_sol[index_ratio]
    df2['solutions'] = best_sol
    return best_ratio_voters, best_sol

best_ratio_voters, best_sol = bruteforce(df2.shape[0])

print("The project selected, with the brute force method, are:",df2[df2['solutions']==1].Project_id)
print("\nThe total cost is:",df2[df2['solutions']==1].Cost.sum())
print("\nThe percentage of satisfied voters is:",best_ratio_voters)

The project selected, with the brute force method, are: 0    proj-03
2    proj-01
Name: Project_id, dtype: object

The total cost is: 227885

The percentage of satisfied voters is: 55.400000000000006


In [208]:
# REVISED SIMPLEX METHOD
nb_projects2 = df2.shape[0]
# Set the coefficients of the linear objective function to be minimized
c2 = -df2['Ratio_voters'].values

# Set the inequality constraint matrix
A2 = [df2.Cost.values.tolist(),]

# Set the inequality constraint vector
max_budget = 500000
b2 = [max_budget]

#Set the bounds
bounds = []
for i in range (nb_projects2):
    bounds.append((0,1))
    
# Select the optimization method
method = 'revised simplex'

# Perform the optimization using the simplex method
res2 = linprog(c2, A_ub=A2, b_ub=b2, method=method, bounds=bounds)

print("The optimal value of the objective function is",round(res2.fun,2))
print("The x array shows us the probability of each project being chosen ", res2.x)
print("If we divide multiply x by r and -1 and we do the sum we have to obtain f :", round(-1*np.sum(res2.x*df2['Ratio_voters']),2))

proj_selected2 = df2['Project_id'].loc[res2.x>=0.5]
print(proj_selected2)

# Total cost of selected projects 
print("\nThe total cost of all the finally selected projects is",sum(df2['Cost'].loc[res2.x>=0.5]))

# Total ratio of satisfied voters 
print("\nThe percentage of satisfied voters is",sum(df2['Ratio_voters'].loc[res2.x>=0.5]))

The optimal value of the objective function is -72.38
The x array shows us the probability of each project being chosen  [1.         0.         1.         0.62832937 0.        ]
If we divide multiply x by r and -1 and we do the sum we have to obtain f : -72.38
0    proj-01
2    proj-03
3    proj-04
Name: Project_id, dtype: object

The total cost of all the finally selected projects is 660962

The percentage of satisfied voters is 82.43


In [211]:
# INTERIOR POINT METHOD
method = 'interior-point'
res_interior2 = linprog(c2, A_ub=A2, b_ub=b2, method=method, bounds=bounds)

# Display the optimization results
print("The optimal value of the objective function is",round(res_interior2.fun,2))
print("The x array shows us the probability of each project being chosen ", res_interior2.x)
print("If we divide multiply x by r and -1 and we do the sum we have to obtain f :", round(-1*np.sum(res_interior2.x*df2['Ratio_voters']),2))

proj_selected2 = df2['Project_id'].loc[res_interior2.x>=0.5]
print(proj_selected2)

# Total cost of selected projects 
print("\nThe total cost of all the finally selected projects is",sum(df2['Cost'].loc[res_interior2.x>=0.5]))

# Total ratio of satisfied voters 
print("\nThe percentage of satisfied voters is",sum(df2['Ratio_voters'].loc[res_interior2.x>=0.5]))

The optimal value of the objective function is -72.38
The x array shows us the probability of each project being chosen  [1.00000000e+00 4.08785190e-14 1.00000000e+00 6.28329373e-01
 2.39610561e-14]
If we divide multiply x by r and -1 and we do the sum we have to obtain f : -72.38
0    proj-01
2    proj-03
3    proj-04
Name: Project_id, dtype: object

The total cost of all the finally selected projects is 660962

The percentage of satisfied voters is 82.43


In [213]:
# CITY HALL OF PARIS METHOD
df2 = df2.sort_values('Nb_votes',ascending= False).reset_index(drop=True)
df2['Cost_cumsum'] = df2.Cost.cumsum().reset_index(drop=True)
list1 = []
list2 = []

for i in range(df2.shape[0]): 
    if df2.Cost_cumsum[i] <=500000:
        list1.append(df2['Project_id'][i])
        list2.append(df2['Ratio_voters'][i])

    else:
        print("We cannot select another project, total cost is",df2['Cost_cumsum'][i-1])
        print("We selected these projects:",list1)
        print("Percentage of satisfied voters is:",round(sum(list2),2))  
        break

We cannot select another project, total cost is 173569
We selected these projects: ['proj-03']
Percentage of satisfied voters is: 36.71


<font color='blue'> 
    - As we can see the Interior point method and the revised simplex method have the same and the best result, but with a problem : the cost is too high. The problem is that the threshold fixed at 0.5 is too low here. 
    
   - The project 4 is accepted with this method because his probability is 0.63 (>0.5) but it's 1 as the other. The Paris Hall of Paris method has the lowest reuslts. And the brute force method has the most optimized result but any other method find the same result.

- Top properly solve this problem with the interior point or the revised simplex method, we could maybe add a constraint of a maximum number of projects selected. Here it would be 2. Or we could increase the threshold like at 0.65.</font> 